In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
tf.enable_eager_execution()
import importlib
import os

from matplotlib import pyplot as plt
%matplotlib notebook

from transliteration import data, train, model_one, script, decode, evaluate

In [2]:
batch_size = 128
our_train_dataset = data.make_dataset('../data/tfrecord/our_2_train.tfrecord',
                                      from_script='en',
                                      to_script='ja',
                                      batch_size=batch_size)
our_valid_dataset = data.make_dataset('../data/tfrecord/our_2_valid.tfrecord',
                                      from_script='en',
                                      to_script='ja',
                                      batch_size=batch_size)
eob_valid_dataset = data.make_dataset('../data/tfrecord/eob_valid.tfrecord',
                                       from_script='en',
                                       to_script='ja',
                                      batch_size=batch_size)
muse_valid_dataset = data.make_dataset('../data/tfrecord/muse_valid.tfrecord',
                                       from_script='en',
                                       to_script='ja',
                                       batch_size=batch_size)

In [3]:
optimizer = tf.train.AdamOptimizer()

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred)
    return tf.reduce_mean(loss_ * mask)

ja_encoder_config = model_one.Config(lstm_size=120,
                                     embedding_size=30,
                                     attention_size=None,
                                     vocab_size=script.SCRIPTS['en'].vocab_size)
ja_decoder_config = model_one.Config(lstm_size=120,
                                     embedding_size=30,
                                     attention_size=60,
                                     vocab_size=script.SCRIPTS['ja'].vocab_size)
ja_encoder = model_one.Encoder(ja_encoder_config)
ja_decoder = model_one.Decoder(ja_decoder_config)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = None

def make_checkpoint_obj():
    return tf.train.Checkpoint(optimizer=optimizer,
                               ja_encoder=ja_encoder,
                               ja_decoder=ja_decoder)

In [4]:
best_val_loss = None
ja_checkpoint = None

In [5]:
for e in range(15):
    loss = train.run_one_epoch(our_train_dataset,
                               True,
                               from_script='en',
                               to_script='ja',
                               encoder=ja_encoder,
                               decoder=ja_decoder,
                               optimizer=optimizer,
                               loss_function=loss_function)
    valid_loss = train.run_one_epoch(our_valid_dataset,
                                     False,
                                     from_script='en',
                                     to_script='ja',
                                     encoder=ja_encoder,
                                     decoder=ja_decoder,
                                     loss_function=loss_function)
    if best_val_loss is None or valid_loss < best_val_loss:
        best_val_loss = valid_loss
        if checkpoint is None:
            checkpoint = make_checkpoint_obj()
        ja_checkpoint = checkpoint.save(file_prefix=checkpoint_prefix)
    print("Epoch {}: Train Loss {:.3f}, Valid Loss {:.3f}".format(e, loss, valid_loss))
    print(decode.transliterate(input_strs=['derick'],
                               from_script='en',
                               to_script='ja',
                               encoder=ja_encoder,
                               decoder=ja_decoder,
                               k_best=2,
                               decoding_method=decode.beam_search_decode))

Instructions for updating:
Colocations handled automatically by placer.


Epoch 0: Train Loss 18.893, Valid Loss 17.194
([['', 'ル']], array([[-4.09070444, -8.1306653 ]]))


Epoch 1: Train Loss 15.474, Valid Loss 14.672
([['', 'ー']], array([[-3.96696663, -7.25935888]]))


Epoch 2: Train Loss 14.238, Valid Loss 13.996
([['ー', 'ン']], array([[-6.95424581, -7.10179305]]))


Epoch 3: Train Loss 13.651, Valid Loss 13.575
([['ーー', 'ーン']], array([[-9.0966785 , -9.29305768]]))


...

Epoch 10: Train Loss 12.644, Valid Loss 13.121
([['リーン', 'リーー']], array([[-10.08886331, -10.18286264]]))


Epoch 11: Train Loss 12.535, Valid Loss 12.360
([['リー', 'ラー']], array([[-9.50044107, -9.51245451]]))


Epoch 12: Train Loss 12.444, Valid Loss 12.439
([['リー', 'リーン']], array([[-9.25696707, -9.55111781]]))


Epoch 13: Train Loss 12.323, Valid Loss 12.569
([['リー', 'リール']], array([[-8.90347123, -9.25400236]]))


Epoch 14: Train Loss 12.290, Valid Loss 12.488
([['リー', 'リン']], array([[-8.347754  , -8.82998943]]))


In [6]:
for e in range(15, 100):
    loss = train.run_one_epoch(our_train_dataset,
                               True,
                               from_script='en',
                               to_script='ja',
                               encoder=ja_encoder,
                               decoder=ja_decoder,
                               optimizer=optimizer,
                               loss_function=loss_function)
    valid_loss = train.run_one_epoch(our_valid_dataset,
                                     False,
                                     from_script='en',
                                     to_script='ja',
                                     encoder=ja_encoder,
                                     decoder=ja_decoder,
                                     loss_function=loss_function)
    if best_val_loss is None or valid_loss < best_val_loss:
        best_val_loss = valid_loss
        if checkpoint is None:
            checkpoint = make_checkpoint_obj()
        ja_checkpoint = checkpoint.save(file_prefix=checkpoint_prefix)
    print("Epoch {}: Train Loss {:.3f}, Valid Loss {:.3f}".format(e, loss, valid_loss))
    print(decode.transliterate(input_strs=['derick'],
                               from_script='en',
                               to_script='ja',
                               encoder=ja_encoder,
                               decoder=ja_decoder,
                               k_best=2,
                               decoding_method=decode.beam_search_decode))

Epoch 15: Train Loss 12.236, Valid Loss 12.349
([['リー', 'リン']], array([[-7.69474316, -8.27618408]]))


Epoch 16: Train Loss 12.216, Valid Loss 12.339
([['リー', 'リン']], array([[-7.87511623, -8.41008997]]))


Epoch 17: Train Loss 12.089, Valid Loss 11.833
([['リー', 'リン']], array([[-7.43134153, -8.0247581 ]]))


Epoch 18: Train Loss 12.037, Valid Loss 11.693
([['リー', 'リン']], array([[-7.06350589, -7.84574306]]))


Epoch 19: Train Loss 11.987, Valid Loss 11.993
([['ロー', 'ロン']], array([[-7.59558558, -8.03289866]]))


...

Epoch 95: Train Loss 2.603, Valid Loss 15.524
([['デック', 'シック']], array([[-3.530673  , -3.74382568]]))


Epoch 96: Train Loss 2.481, Valid Loss 15.711
([['デック', 'シール']], array([[-3.01422212, -3.3645761 ]]))


Epoch 97: Train Loss 2.376, Valid Loss 14.509
([['シール', 'シック']], array([[-3.18444835, -3.29695437]]))


Epoch 98: Train Loss 2.288, Valid Loss 17.458
([['シール', 'シルク']], array([[-3.2040583 , -3.24740525]]))


Epoch 99: Train Loss 2.151, Valid Loss 16.061
([['デック', 'シック']], array([[-2.91614201, -3.34366792]]))


In [7]:
checkpoint.restore(ja_checkpoint).assert_consumed()
train.run_one_epoch(our_valid_dataset,
                    False,
                    from_script='en',
                    to_script='ja',
                    encoder=ja_encoder,
                    decoder=ja_decoder,
                    loss_function=loss_function)

<tf.Tensor: id=13781081, shape=(), dtype=float32, numpy=11.939638>

In [8]:
train.run_one_epoch(eob_valid_dataset,
                    False,
                    from_script='en',
                    to_script='ja',
                    encoder=ja_encoder,
                    decoder=ja_decoder,
                    loss_function=loss_function)

<tf.Tensor: id=13927716, shape=(), dtype=float32, numpy=41.722157>

In [9]:
train.run_one_epoch(muse_valid_dataset,
                    False,
                    from_script='en',
                    to_script='ja',
                    encoder=ja_encoder,
                    decoder=ja_decoder,
                    loss_function=loss_function)

<tf.Tensor: id=13995484, shape=(), dtype=float32, numpy=21.196993>

In [10]:
valid_df = pd.read_csv('../data/split/muse_pairs_valid.csv',
                       keep_default_na=False)

In [11]:
tr = decode.transliterate(input_strs=valid_df['en'].values,
                          from_script='en',
                          to_script='ja',
                          encoder=ja_encoder,
                          decoder=ja_decoder,
                          k_best=20,
                          num_beams=40,
                          decoding_method=decode.beam_search_decode)

In [12]:
evaluate.top_k_accuracy(valid_df['ja'].values, tr, k=1)

0.004070556309362279

In [13]:
evaluate.top_k_accuracy(valid_df['ja'].values, tr, k=10)

0.012211668928086838